In [ ]:
# imports
import pandas as pd
import numpy as np
import sys
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer


In [ ]:
# labeled data
Dl=pd.read_csv('labeled_data.csv')
# shuffle data
Dl=Dl.sample(frac=1)
Dl

In [ ]:
# unlabeled data
Du=pd.read_csv("unlabeled_data.csv")
# shuffle data
Du=Du.sample(frac=1)
Du

In [ ]:
# merge the data to train the vectorizer
all=pd.merge(Dl,Du,how='outer',on=['ad'])
# tfidf vectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(all['ad'])


In [ ]:
def RKM_SSL(data_vectorizer,Dl,Du,final_clusters_data,num_classes,th=10):
      '''
          Recursive K-Means for Semi-Supervised Learning
          params :
              data_vectoriwer : a trained vectorizer
              Dl: labeled data
              Du: unlabeled data
              final_clusters_data: the data structure grouping the data labeled using the clustering
              num_classes: the number of clusters
              th : the proportion of the minimum class with respect to the maximum class in a partition
          returns :
              final_clusters: the data structure grouping the data labeled with the clustering final_clusters['labels'] contain labels ,
      '''

      #the data structure regrouping labeled data and data to label using clustering
      final_clusters=final_clusters_data
      # the partition is divisible ?
      branchement=False
      Dt=Dl
      Dt=Dt.append(Du)

      #initialization
      k=num_classes
      # randomly chosen 2 samples (one from each unique class)
      random_sample=Dl[Dl['label']==1.0].sample(1).append(Dl[Dl['label']==0.0].sample(1))
      # text seeds
      initial_seeds=random_sample['ad']
      seed_labels=random_sample['label']
      # vectorizer
      vectorizer = data_vectorizer
      # feature extraction
      features=vectorizer.transform(Dt['ad']).toarray()
      # vectorized_seeds
      initial_seeds=vectorizer.transform(initial_seeds).toarray()
      #Initial K-means clustering
      # Apply K-means on DT with IntialSeeds as initial cluster centroids to get K Partition
      kmeans = KMeans(n_clusters=2, random_state=0,init=initial_seeds,n_init=1).fit(features)
      Dt['kmeans.labes']=kmeans.labels_

      #Recursive K-means clustering
      for i in range(k):

          branchement=False
          # select one partition at a time to work on
          partition=Dt.loc[Dt['kmeans.labes']==i]
          # unique labels per partition
          NCP=partition['label'].nunique()
          # number of samples in each class
          LSP=[]
          LSP.append(partition[partition['label']==0].shape[0])
          LSP.append(partition[partition['label']==1].shape[0])

          # the majority class
          Class_label=np.argmax(LSP)

          # check if the partition has more the one class

          if NCP >1:

            other_class= partition[partition['kmeans.labes']==np.argmin(LSP)]

            relative_percentage=(LSP[np.argmin(LSP)]/LSP[Class_label])*100
            if(relative_percentage==100):
              if(LSP[np.argmin(LSP)]==LSP[Class_label]==1):
                  relative_percentage=th

            if relative_percentage >th :

              labeled=partition[partition['label']>=0].drop(['kmeans.labes'], axis = 1)
              unlabeled=partition[partition['label'].isna()].drop(['kmeans.labes','label'], axis = 1)
              # the partition is divisble
              branchement=True
              # get the final cluster
              final_clusters=RKM_SSL(vectorizer,labeled,unlabeled,final_clusters,num_classes=NCP)




          # update final clusters with the partition
          if(branchement==False):
                partition=partition.drop(['kmeans.labes'], axis = 1)
                final_clusters=final_clusters.append(partition)



          #Labeling unlabeled samples
          final_clusters.loc[final_clusters['label'].isna(),'labels_cluster']=1 # label_cluster=1 indicates that the ad is labeled using clustering
          final_clusters.loc[final_clusters['label'].isna(),'label'] =Class_label # label refers to the label assigned to the ad



      return final_clusters



In [ ]:
sys.setrecursionlimit(10000)


In [ ]:
# pick a sample of the unlabeled data
Du_ = Du.iloc[:1200]
# create the data structure regrouping labeled data and data to label using clustering
column_names = ["ad","label","labels_cluster"]
final_clusters = pd.DataFrame(columns =column_names)
results=RKM_SSL(vectorizer,Dl,Du_,final_clusters,num_classes=2,th=10)


In [ ]:
results.to_csv("results_clustering.csv")